<a href="https://colab.research.google.com/github/PrajwalPPatil/Smart-Work-Force-Allocation-For-Facility-Management_-Power-BI-/blob/main/Smart_WorkForce_Allocation_for_Facility_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Smart WorkForce Allocation For Facility Management**

In [ ]:
# Load the uploaded dataset to inspect its structure and prepare for further analysis
import pandas as pd

# Load the dataset
df = pd.read_excel("/content/simulated_facility_workforce_data.xlsx")

# Display basic info and first few rows
df.info(), df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2307 entries, 0 to 2306
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  2307 non-null   datetime64[ns]
 1   Employee_ID           2307 non-null   int64         
 2   Shift_Start           2307 non-null   object        
 3   Shift_End             2307 non-null   object        
 4   Location              2307 non-null   object        
 5   Skill_Level           2307 non-null   object        
 6   Job_Role              2307 non-null   object        
 7   Workload_Demand_Site  2307 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 144.3+ KB


(None,
         Date  Employee_ID Shift_Start Shift_End Location Skill_Level  \
 0 2025-01-01           45    14:00:00  22:00:00   Site A       Basic   
 1 2025-01-01           34    06:00:00  14:00:00   Site A    Advanced   
 2 2025-01-01           30    14:00:00  22:00:00   Site A       Basic   
 3 2025-01-01          100    22:00:00  06:00:00   Site A       Basic   
 4 2025-01-01           70    06:00:00  14:00:00   Site A    Advanced   
 
      Job_Role  Workload_Demand_Site  
 0     Cleaner                     8  
 1     Cleaner                     8  
 2     Cleaner                     8  
 3  Supervisor                     8  
 4  Technician                     8  )

In [ ]:
# Check data shape
df.shape

(2307, 8)

In [ ]:
# Check data columns
df.columns

Index(['Date', 'Employee_ID', 'Shift_Start', 'Shift_End', 'Location',
       'Skill_Level', 'Job_Role', 'Workload_Demand_Site'],
      dtype='object')

In [ ]:
df['Location'].value_counts()

,count
Location,
Site A,613
Site D,604
Site C,568
Site B,522


In [ ]:
df['Skill_Level'].value_counts()

,count
Skill_Level,
Basic,1270
Intermediate,639
Advanced,398


In [ ]:
df['Job_Role'].value_counts()

,count
Job_Role,
Cleaner,816
Supervisor,785
Technician,706


In [ ]:
# Check Missing_values
missing_values=df.isnull().sum()
print("\nMissing values in each column:")
print(missing_values)


Missing values in each column:
Date                    0
Employee_ID             0
Shift_Start             0
Shift_End               0
Location                0
Skill_Level             0
Job_Role                0
Workload_Demand_Site    0
dtype: int64


✅ Data-Driven Optimization of Manpower Deployment


In [ ]:
#Use pandas to analyze deployment across sites and shifts:
import pandas as pd

# Load your dataset
df = pd.read_excel('/content/simulated_facility_workforce_data.xlsx')
df['Date'] = pd.to_datetime(df['Date'])

# Manpower per site and shift
deployment_summary = df.groupby(['Date', 'Location', 'Shift_Start', 'Shift_End']).agg({
    'Employee_ID': 'count',
    'Workload_Demand_Site': 'first'
}).reset_index().rename(columns={'Employee_ID': 'Assigned_Employees'})

deployment_summary['Staffing_Gap'] = deployment_summary['Assigned_Employees'] - deployment_summary['Workload_Demand_Site']

✅ Identify Overstaffing/Understaffing Patterns

In [ ]:
# Identify patterns
over_under_stats = deployment_summary.groupby('Location')['Staffing_Gap'].agg(['mean', 'std', 'min', 'max'])

# Highlight problem days
problem_days = deployment_summary[deployment_summary['Staffing_Gap'].abs() > 2]

✅ Predictive Model for Staff Requirements

In [ ]:
# We'll use date + location features to forecast required staffing levels.
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Feature engineering
deployment_summary['DayOfWeek'] = deployment_summary['Date'].dt.dayofweek
deployment_summary['Month'] = deployment_summary['Date'].dt.month
deployment_encoded = pd.get_dummies(deployment_summary, columns=['Location'])

X = deployment_encoded[['DayOfWeek', 'Month'] + [col for col in deployment_encoded.columns if col.startswith('Location_')]]
y = deployment_encoded['Workload_Demand_Site']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Model
model = LinearRegression()
model.fit(X_train, y_train)
deployment_summary['Predicted_Demand'] = model.predict(X)

# Save predictions
deployment_summary.to_csv('/content/predicted_staff_demand.csv', index=False)

✅ Optimized Shift Allocation Recommendation

In [ ]:
# Identify days with surplus staff and suggest reallocation:
# Example: reassign overstaffed employees to understaffed locations on same day
shift_balance = deployment_summary.pivot_table(index='Date', columns='Location', values='Staffing_Gap', aggfunc='sum').fillna(0)

# Suggest improvements
suggestions = shift_balance.apply(lambda row: 'Redistribute' if row.max() > 2 and row.min() < -2 else 'OK', axis=1)
redistribution_days = shift_balance[suggestions == 'Redistribute']

In [ ]:
# Check First Five Rows
df.head()

,Date,Employee_ID,Shift_Start,Shift_End,Location,Skill_Level,Job_Role,Workload_Demand_Site
0,2025-01-01,45,14:00:00,22:00:00,Site A,Basic,Cleaner,8
1,2025-01-01,34,06:00:00,14:00:00,Site A,Advanced,Cleaner,8
2,2025-01-01,30,14:00:00,22:00:00,Site A,Basic,Cleaner,8
3,2025-01-01,100,22:00:00,06:00:00,Site A,Basic,Supervisor,8
4,2025-01-01,70,06:00:00,14:00:00,Site A,Advanced,Technician,8
